In [1]:
!ls

2010 Federal STEM Education Inventory Data Set.xls
Untitled.ipynb
task.txt


In [2]:
!cat task.txt

Create a public repo called ml_stem_analysis on github and share with us.
 
Data: https://data.world/education/2010-federal-stem-inventory/file/2010%20Federal%20STEM%20Education%20Inventory%20Data%20Set.xls
 
Stage 1:
1) Calculate % growth of funding between year 2008 & 2009.
2) If funding is positive, tag it as 1, if funding is negative tag it as 0. This is the target variable.
 
Stage 2:
1) Create graphs of univariate distribution of all non funding variables and share on a jupyter notebook. Just FYI - Funding FY2008, FY2009, FY2010 are the "funding variables"
2) Calculate mutual_info_score of target variable created in stage 1 & ALL non funding variables and share on a jupyter notebook.
 
Stage 3:
1) Divide data into train & test samples. (70-30 split)
2) Select features & build xgboost model. You will be judged on roc_auc_score on test sample.
3) Write testcases on all user defined functions using pytest framework. This is one of the most important steps of this interview.


In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_excel('2010 Federal STEM Education Inventory Data Set.xls')

In [5]:
df.shape

(254, 255)

In [6]:
from IPython.display import display
with pd.option_context('display.max_rows', None, 'display.max_columns', 10):
    display(df.head(3))

,PROGRAM,Unnamed: 1,Unnamed: 2,Unnamed: 3,FUNDING,...,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254
Index Number,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,...,Y4) Portfolio Evaluation/Review Type,NaN,NaN,Y5) Expert Review Type,NaN
PRG_0001,Centers for Ocean Sciences Education Excellence ~,National Science Foundation,Directorate for Geosciences (GEO),The Division of Ocean Sciences seeks to establ...,2002,...,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN
PRG_0002,Broadening Participation in Computing (BPC),National Science Foundation,Directorate for Computer & Information Science...,The BPC Program was not primarily aimed at edu...,2006,...,NaN,NaN,NO,NaN,NaN


In [7]:
df.columns = df.iloc[0]
df.head(3)

Index Number,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,...,nan,nan,Y3) Process or Implementation Evaluation Type,nan,nan,Y4) Portfolio Evaluation/Review Type,nan,nan,Y5) Expert Review Type,nan
Index Number,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,...,NaN,NaN,Y3) Process or Implementation Evaluation Type,NaN,NaN,Y4) Portfolio Evaluation/Review Type,NaN,NaN,Y5) Expert Review Type,NaN
PRG_0001,Centers for Ocean Sciences Education Excellence ~,National Science Foundation,Directorate for Geosciences (GEO),The Division of Ocean Sciences seeks to establ...,2002,5.24,7.19,5.7,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN
PRG_0002,Broadening Participation in Computing (BPC),National Science Foundation,Directorate for Computer & Information Science...,The BPC Program was not primarily aimed at edu...,2006,14,14,14,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN


In [8]:
df = df.iloc[1:]

In [9]:
df.columns

Index([                              'Investment Name',
                                              'Agency',
                                           'Subagency',
                                'A) Brief Description',
                                 'B) Year Established',
                                  'C1) Funding FY2008',
                                  'C2) Funding FY2009',
                                  'C3) Funding FY2010',
                'D) Mission-specific or General STEM?',
       'E) Agency or Mission-Related Workforce Needs?',
       ...
                                                   nan,
                                                   nan,
       'Y3) Process or Implementation Evaluation Type',
                                                   nan,
                                                   nan,
                'Y4) Portfolio Evaluation/Review Type',
                                                   nan,
                                     

In [10]:
# Create a list for df columns as shown in excel (here, multiple spanning of columns will be shown as colname + 0/1/2....)
lis = []
k = 0
for x, y in zip(df.columns.isnull(), df.columns):
    if not x:
        z = y
        lis.append(y)
        j = 0
        k = 0
    else:
        if j == 0:
            lis.pop()
            lis.append(z + str(k))
            k += 1
            lis.append(str(z) + str(k))
        else:
            k += 1
            lis.append(str(z) + str(k))
lis[:15]

['Investment Name',
 'Agency',
 'Subagency',
 'A) Brief Description',
 'B) Year Established',
 'C1) Funding FY2008',
 'C2) Funding FY2009',
 'C3) Funding FY2010',
 'D) Mission-specific or General STEM?',
 'E) Agency or Mission-Related Workforce Needs?',
 'F1) Primary Investment Objective',
 'F2) Secondary Investment Objectives0',
 'F2) Secondary Investment Objectives1',
 'F2) Secondary Investment Objectives2',
 'F2) Secondary Investment Objectives3']

In [11]:
df.columns = lis

In [12]:
df.head(3)

,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,...,Y2) Summative Evaluation Type1,Y2) Summative Evaluation Type2,Y3) Process or Implementation Evaluation Type0,Y3) Process or Implementation Evaluation Type1,Y3) Process or Implementation Evaluation Type2,Y4) Portfolio Evaluation/Review Type0,Y4) Portfolio Evaluation/Review Type1,Y4) Portfolio Evaluation/Review Type2,Y5) Expert Review Type0,Y5) Expert Review Type1
PRG_0001,Centers for Ocean Sciences Education Excellence ~,National Science Foundation,Directorate for Geosciences (GEO),The Division of Ocean Sciences seeks to establ...,2002,5.24,7.19,5.7,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN
PRG_0002,Broadening Participation in Computing (BPC),National Science Foundation,Directorate for Computer & Information Science...,The BPC Program was not primarily aimed at edu...,2006,14,14,14,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN
PRG_0003,CISE Pathways to Revitalized Undergraduate Com...,National Science Foundation,Directorate for Computer & Information Science...,Through the CISE Pathways to Revitalized Under...,2007,5,5,4.37,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN


## Task 1
